In [ ]:
from autumn.tools.project import get_project
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from autumn.tools.plots.utils import REF_DATE
from autumn.models.covid_19.constants import BASE_DATETIME
from autumn.tools.calibration.targets import get_target_series
import pandas as pd
from autumn.tools.utils.pretty import pretty_print
from summer.utils import ref_times_to_dti

In [ ]:
region = "barwon-south-west"
project = get_project("covid_19", region)

In [ ]:
custom_params = project.param_set.baseline
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()

In [ ]:
outputs = (
    "notifications", 
    "infection_deaths", 
    "hospital_admissions",
    "hospital_occupancy",
    "icu_admissions", 
    "icu_occupancy"
)

In [ ]:
target_dates, target_values = get_target_series(project.calibration.targets, REF_DATE, "hospital_occupancy")

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(12, 15), sharex="all")
fig.tight_layout(w_pad=1.5, h_pad=3.5)

for i_ax, axis in enumerate(axes.reshape(-1)):
    out = outputs[i_ax]
    target_dates, target_values = get_target_series(project.calibration.targets, REF_DATE, outputs[i_ax])
    axis.plot(ref_times_to_dti(REF_DATE, model.times), derived_df[outputs[i_ax]])
    axis.plot(ref_times_to_dti(REF_DATE, model.times), derived_df[outputs[i_ax]])
    axis.scatter(target_dates, target_values, c="k")
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_title(out)